In [ ]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import frauddetection as fd
import sourcedata as sd
import countrymanagement as cm
import mccmanagement as mccm
reload(fd)
reload(sd)
reload(cm)
reload(mccm)

print('done')

In [ ]:
import pandas as pd

#For Kaggle
#date=''

#For WL data
date='20241118'
source='WL'

saveImg=False

print('done')

dfTrxSaved = pd.read_csv('../data/cleaned/'+source+'export'+date+'.csv')
dfTrxSaved.head(5)

In [ ]:
dfTrxSaved['term_mcc'] = dfTrxSaved['term_mcc'].astype(str)
dfTrxSaved['word']=dfTrxSaved['term_country']+dfTrxSaved['term_mcc']

In [ ]:
sorted_df = dfTrxSaved.sort_values(by=['card_pan_id','trx_date_time'])
sorted_df=sorted_df.drop(columns=['mcd_fraud_score','vaa_score',
                                 'trx_amount_log10','card_brand_MCD',	'card_brand_VIS','country_group_BELGIUM','country_group_EUROPE',
                                 'clusterCardHolder_cluster0',	'clusterCardHolder_cluster1',	'clusterCardHolder_cluster2',	'clusterCardHolder_cluster3',	
                                 'clusterCardHolder_cluster4',	'clusterMerchant_M0',	'clusterMerchant_M1','clusterMerchant_M2','clusterMerchant_M3',
                                 'clusterMerchant_UNKNOWN',
                                  'country_group_WORLD','mcc_group_CLOTHING_STORES',	
                                  'mcc_group_GOUVERNEMENT_SERVICES','mcc_group_OTHER','mcc_group_PROFESSIONAL_SERVICES',	
                                  'mcc_group_RETAIL_OUTLET_SERVICES','mcc_group_UTILITY_SERVICES','mcc_group_ATM',	
                                  'trx_reversal_FULL_REVERSAL',	'trx_reversal_NO_RESERSAL',	
                                  'trx_reversal_PARTIAL_REVERSAL','clusterCardHolder_UNKNOWN'
                                 ])



sorted_df['card_pan_id1'] = sorted_df['card_pan_id'].shift(-1)
#sorted_df['term_mcc1']= sorted_df['term_mcc'].shift(-1) if sorted_df['card_pan_id1'] == sorted_df['card_pan_id'] else 'XXXX'
sorted_df['word1'] = sorted_df['word'].shift(-1)
sorted_df['ecom1'] = sorted_df['ecom'].shift(-1)
sorted_df['db_uuid1'] = sorted_df['db_uuid'].shift(-1)
sorted_df.head(10)


In [ ]:
sorted_df1=sorted_df[(sorted_df['card_pan_id']==sorted_df['card_pan_id1'])]
sorted_df0=sorted_df[(sorted_df['card_pan_id']!=sorted_df['card_pan_id1'])]

print(sorted_df.shape)
print(sorted_df0.shape)
print(sorted_df1.shape)

print(sorted_df.Class.value_counts(normalize=True))
print(sorted_df0.Class.value_counts(normalize=True))
print(sorted_df1.Class.value_counts(normalize=True))


#sorted_df4.head(50)

In [ ]:
import numpy as np

dfTrxSaved['previous_trx']=0

sorted_dfTemp=sorted_df[(sorted_df['card_pan_id']==sorted_df['card_pan_id1'])]
dfTrxSaved['previous_trx']=np.where(sorted_df.index.isin(sorted_dfTemp.index),1,dfTrxSaved['previous_trx'])

dfTrxSaved['card_pan_id1']=sorted_df['card_pan_id1']
dfTrxSaved['word1'] = sorted_df['word1']
dfTrxSaved['ecom']=sorted_df['ecom']
dfTrxSaved['ecom1']=sorted_df['ecom1']
dfTrxSaved['db_uuid1']=sorted_df['db_uuid1']

print(dfTrxSaved['previous_trx'].value_counts())
dfTrxSaved.head()


In [ ]:
temp1=dfTrxSaved[(dfTrxSaved['card_pan_id']==dfTrxSaved['card_pan_id1'])&(dfTrxSaved['word']!=dfTrxSaved['word1'])]
temp2=dfTrxSaved[(dfTrxSaved['card_pan_id']==dfTrxSaved['card_pan_id1'])&(dfTrxSaved['word']==dfTrxSaved['word1'])]

print(temp1.shape)
print(temp2.shape)
temp2[['word','word1']].head()

In [ ]:
temp2=dfTrxSaved[(dfTrxSaved['card_pan_id']==dfTrxSaved['card_pan_id1'])&(dfTrxSaved['word']!=dfTrxSaved['word1'])]
temp2[['trx_date_time','card_pan_id','card_pan_id1','word','word1']]

In [ ]:
import pandas as pd
import os
f=open("../data/processed/countrymcc.txt","w+") # file name and mode
text=''
for index,row in temp2.iterrows():
    if((index % 5000)==0):
        print("",index)
    f.write(row['word1']+" "+row['word']+'.\n')
    text=text+row['word1']+" "+row['word']+'.\n'
f.close()
print('done ',index)

In [ ]:
import nltk
#nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
token=word_tokenize(text)
tokenClean= [i for i in token if i is not '.']
freq= nltk.FreqDist(tokenClean)
freq.plot(20)

In [ ]:
import gensim
from gensim.models import Word2Vec

from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
 
warnings.filterwarnings(action='ignore')

data = []
 
# iterate through each sentence in the file
for i in sent_tokenize(text):
    temp = []
 
    # tokenize the sentence into words
    for j in word_tokenize(i):
        temp.append(j)
 
    data.append(temp)

model = Word2Vec(data,min_count=1,window=2,max_vocab_size=2000000)
print('done')

In [ ]:
def word_not_present(wordT):
    if (wordT in model.wv.key_to_index):
        return wordT
    else:
        print(wordT)
        return 'BEL5411'


dfTrxSaved['word'] = dfTrxSaved['word'].apply(word_not_present)
dfTrxSaved['word1'] = dfTrxSaved['word1'].apply(word_not_present)

In [ ]:
#dfTrxSaved['distancePrevTrx']= 
#model.wv.similarity(dfTrxSaved['word'], dfTrxSaved['word1'])
print(dfTrxSaved[['word','word1']].head(128))
dfTrxSaved['distancePrevTrx']= dfTrxSaved[['word1','word']].apply(lambda x: model.wv.similarity(x[0],x[1]), axis=1)

#dfTemp=dfTrxSaved[(dfTrxSaved['previous_trx']==1)


#dfTrxSaved['distancePrevTrx']=np.where(sorted_df.index.isin(dfTemp.index),1,model.wv.similarity(dfTrxSaved['word'], dfTrxSaved['word1']))

In [ ]:
print(dfTrxSaved['distancePrevTrx'][(dfTrxSaved['distancePrevTrx']<1.0)].shape)
print(dfTrxSaved['distancePrevTrx'][(dfTrxSaved['distancePrevTrx']<0.9)].shape)
print(dfTrxSaved['distancePrevTrx'][(dfTrxSaved['distancePrevTrx']<0.8)].shape)
print(dfTrxSaved['distancePrevTrx'][(dfTrxSaved['distancePrevTrx']<0.7)].shape)
print(dfTrxSaved['distancePrevTrx'][(dfTrxSaved['distancePrevTrx']<0.6)].shape)
print(dfTrxSaved['distancePrevTrx'][(dfTrxSaved['distancePrevTrx']<0.5)].shape)
print(dfTrxSaved['distancePrevTrx'][(dfTrxSaved['distancePrevTrx']<0.4)].shape)
print(dfTrxSaved['distancePrevTrx'][(dfTrxSaved['distancePrevTrx']<0.3)].shape)
print(dfTrxSaved['distancePrevTrx'][(dfTrxSaved['distancePrevTrx']<0.2)].shape)
print(dfTrxSaved['distancePrevTrx'][(dfTrxSaved['distancePrevTrx']<0.1)].shape)

print(dfTrxSaved['Class'][(dfTrxSaved['distancePrevTrx']==1.0)].value_counts(normalize=True))
print(dfTrxSaved['Class'][(dfTrxSaved['distancePrevTrx']<1.0)].value_counts(normalize=True))
print(dfTrxSaved['Class'][(dfTrxSaved['distancePrevTrx']<0.9)].value_counts(normalize=True))
print(dfTrxSaved['Class'][(dfTrxSaved['distancePrevTrx']<0.8)].value_counts(normalize=True))
print(dfTrxSaved['Class'][(dfTrxSaved['distancePrevTrx']<0.7)].value_counts(normalize=True))
print(dfTrxSaved['Class'][(dfTrxSaved['distancePrevTrx']<0.6)].value_counts(normalize=True))
print(dfTrxSaved['Class'][(dfTrxSaved['distancePrevTrx']<0.5)].value_counts(normalize=True))
print(dfTrxSaved['Class'][(dfTrxSaved['distancePrevTrx']<0.4)].value_counts(normalize=True))
print(dfTrxSaved['Class'][(dfTrxSaved['distancePrevTrx']<0.3)].value_counts(normalize=True))
print(dfTrxSaved['Class'][(dfTrxSaved['distancePrevTrx']<0.2)].value_counts(normalize=True))

In [ ]:
import numpy as np
max = 11

print('distancePrevTrx')
for bins in np.arange(2,max,1):
    dfTrxSaved['distancePrevTrxBin']=pd.cut(dfTrxSaved['distancePrevTrx'], bins=bins)
    print(f'IV trx_local_amt_val bins={bins} {fd.calc_iv(dfTrxSaved,'distancePrevTrxBin','Class',pr=0)}')